# Explorando as estatísticas de comércio exterior brasileiras

As bases de dados exportações e importações brasileiras podem ser encontradas no seguinte endereço:

https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta

Observe que na primeira seção - Base de dados detalhada por NCM - encontramos links para os arquivos .csv para cada ano.

Crie uma função que gere uma lista com os urls de todos os anos disponíveis (1997 a 2023), tanto para importação quanto para exportação.

In [1]:
def cria_url(fluxo, ano):
    url_base = "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/"
    return f"{url_base}{fluxo.upper()}_{str(ano)}.csv"

In [ ]:
urls_balanca = [cria_url(f, a) for f in ["exp", "imp"] for a in range(1997, 2024)]

Aproveitando os códigos da aula, baixe e salve o arquivo de exportações de 1997.

In [5]:
import os
import requests

def build_path(folder):
    current_folder = os.getcwd()
    folderpath = os.path.join(current_folder, folder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): os.makedirs(folderpath)
    return folderpath


# folder é uma subpasta do seu diretório atual
# o nome do arquivo é gerado automaticamente
def download_save_unverified(url, folder):
    
    folderpath = build_path(folder)
    filepath = os.path.join(folderpath, url.split('/')[-1])

    # bypass da validação para evitar SSLCertVerificationError
    response = requests.get(url, verify = False) 

    if response.status_code == 200:
        content = response.content

        with open(filepath, mode='wb') as csvfile:
            csvfile.write(content)

In [ ]:
download_save_unverified(urls_balanca[0], "data/arquivos_exp")

Importe a função **read_csv** da biblioteca **pandas** e tente abrir o arquivo baixado. 

In [ ]:
from pandas import read_csv

read_csv("data/arquivos_exp/EXP_1997.csv")

Algo estranho na formatação da tabela? Investigue a primeira linha do arquivo baixado e tente ajustar algum parâmetro da função **read_csv**.

In [ ]:
import csv
filepath = "data/arquivos_exp/EXP_1997.csv"

with open(filepath) as csvfile:
    lines = csvfile.readlines()
    
lines[:1]

In [ ]:
read_csv("data/arquivos_exp/EXP_1997.csv", sep = ";")

# IBGE - Frequência de nomes


A API do IBGE permite a consulta, com base em dados do Censo, da frequência de nomes por sexo, localidade e década de nascimento. Consulte sua documentação em:

https://servicodados.ibge.gov.br/api/docs/nomes?versao=2


Crie uma função que busque a frequência de um dado nome, agrupando por UF, e retorne um dicionário onde a chave é o nome da UF e o valor é a frequência do nome. Inclua um parâmetro na função para alterar a frequência pela proporção (frequência por 100.000 habitantes).

*dica*: você pode consultar a correspondência entre código e nome da UF pelo endpoint "https://servicodados.ibge.gov.br/api/v1/localidades/estados".

In [7]:
"""
workaround para contornar a exceção: \
    SSLError: [SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] \
    unsafe legacy renegotiation disabled (_ssl.c:997)

é possível considerando que estamos tratando apenas de dados públicos
"""

import requests, urllib3, ssl

class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)
def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

NameError: name 'url' is not defined

In [9]:
estados = get_legacy_session().get("https://servicodados.ibge.gov.br/api/v1/localidades/estados").json()
codigo_estado = {str(i.get("id")): i.get("nome") for i in estados}

In [10]:
def cria_ranking_nome_estado(nome, metrica = "frequencia", codigo_estado):
    nome_query = f"https://servicodados.ibge.gov.br/api/v2/censos/nomes/{nome}?groupBy=UF"
    resp = get_legacy_session().get(nome_query).json()
    ranking = {codigo_estado[uf.get("localidade")]: uf.get("res")[0].get(metrica) 
               for uf in resp}
    
    return ranking

SyntaxError: non-default argument follows default argument (3996331842.py, line 1)